In [1]:
import logging
from neo4j import GraphDatabase
from neo4j.exceptions import ServiceUnavailable
from facebook_scraper import get_posts
class Neo4jService:
    
    
    def __init__(self, uri, user, password):
        self._driver = GraphDatabase.driver(uri, auth=(user, password))

    def close(self):
        self._driver.close()
        
    def crear_Alcaldes(self, tx, nombre):
        tx.run("CREATE (:Alcaldes {nombre: $nombre})", nombre=nombre)
    
    def crear_alcalde(self, tx, nombre):
        tx.run("CREATE (:Alcalde {nombre: $nombre})", nombre=nombre)
        
    def crear_publicacion(self, tx, nombre):
        tx.run("CREATE (:Publicacion {nombre: $nombre})", nombre=nombre)
    
    def crear_fecha(self, tx, nombre):
        tx.run("CREATE (:Fecha {nombre: $nombre})", nombre=nombre)
        
    def crear_texto(self, tx, nombre):
        tx.run("CREATE (:Texto {nombre: $nombre})", nombre=nombre)
        
    def crear_likes(self, tx, nombre):
        tx.run("CREATE (:Likes {nombre: $nombre})", nombre=nombre)
        
    def crear_comentarios(self, tx, nombre):
        tx.run("CREATE (:Comentarios {nombre: $nombre})", nombre=nombre)
        
    def crear_relacion_Alcs_Alc(self, tx, nombre1, nombre2):
        tx.run("MATCH (a:Alcaldes {nombre: $nombre1}) "
               "MATCH (b:Alcalde {nombre: $nombre2}) "
               "MERGE (a)-[:Alcaldess_Alcalde]->(b)",
               nombre1=nombre1, nombre2=nombre2)
        
    def crear_relacion_alc_publicacion(seelf, tx, nombre1, nombre2):
        tx.run("MATCH (a:Alcalde {nombre: $nombre1}) "
              "MATCH (b:Publicacion {nombre: $nombre2}) "
              "MERGE (a)-[:Alcalde_Publicacion]->(b)",
              nombre1=nombre1, nombre2=nombre2)
        
    def crear_relacion_publi_fecha(self, tx, nombre1, nombre2):
        tx.run("MATCH (a:Publicacion {nombre: $nombre1}) "
               "MATCH (b:Fecha {nombre: $nombre2}) "
               "MERGE (b)-[:Publicacion_Fecha]->(a)",
               nombre1=nombre1, nombre2=nombre2)
        
    def crear_relacion_public_texto(self, tx, nombre1, nombre2):
        tx.run("MATCH (a:Publicacion {nombre: $nombre1}) "
               "MATCH (b:Texto {nombre: $nombre2}) "
               "MERGE (b)-[:Publicacion_Texto]->(a)",
               nombre1=nombre1, nombre2=nombre2)
    
    def crear_relacion_public_like(self, tx, nombre1, nombre2):
        tx.run("MATCH (a:Publicacion {nombre: $nombre1}) "
               "MATCH (b:Likes {nombre: $nombre2}) "
               "MERGE (b)-[:Publicacion_Like]->(a)",
               nombre1=nombre1, nombre2=nombre2)
    
    def crear_relacion_public_comen(self, tx, nombre1, nombre2):
        tx.run("MATCH (a:Publicacion {nombre: $nombre1}) "
               "MATCH (b:Comentarios {nombre: $nombre2}) "
               "MERGE (b)-[:Publicacion_Comentarios]->(a)",
               nombre1=nombre1, nombre2=nombre2)
    
        
neo4j = Neo4jService('bolt://localhost:7474', 'neo4j', 'admin')
with neo4j._driver.session() as session:
        contador = -1
        session.write_transaction(neo4j.crear_Alcaldes, "Alcaldes")
        session.write_transaction(neo4j.crear_alcalde, "Pedro Palacios")
        session.write_transaction(neo4j.crear_relacion_Alcs_Alc , "Alcaldes" , "Pedro Palacios")
        for post in get_posts ('jorgeyundamachado', pages=52, timeout=1):
            contador = contador + 1
            publicacion = "publicaciones de PALACIOS N°" +str(contador)
            fecha = "Fecha de la publicacion N°" +str(contador)+ " es " +str(post['time'])
            text = "El texto de la publicacion N°" +str(contador)+ " es " +str(post['text'])
            likes = "Los likes de la publicacion N°" +str(contador)+ " es " +str(post['likes'])
            comentarios = "El numero de comentarios de la publicacion N°" +str(contador)+ " es " +str(post['comments'])
            session.write_transaction(neo4j.crear_publicacion, publicacion)
            session.write_transaction(neo4j.crear_fecha, fecha)
            session.write_transaction(neo4j.crear_texto, text)
            session.write_transaction(neo4j.crear_likes, likes)
            session.write_transaction(neo4j.crear_comentarios, comentarios)
            session.write_transaction(neo4j.crear_relacion_alc_publicacion, "Jorge Yunda", publicacion)
            session.write_transaction(neo4j.crear_relacion_publi_fecha, publicacion, fecha)
            session.write_transaction(neo4j.crear_relacion_public_texto, publicacion, text)
            session.write_transaction(neo4j.crear_relacion_public_like, publicacion, likes)
            session.write_transaction(neo4j.crear_relacion_public_comen, publicacion, comentarios)
            print(publicacion)

ServiceUnavailable: Cannot to connect to Bolt service on IPv4Address(('localhost', 7474)) (looks like HTTP)